<h1>Chapter 16. Overload of Operators</h1>

**Operator Overloading** in Python allows user-defined classes to redefine the behavior of built-in operators (e.g., `+`, `-`, `*`, etc.) when applied to their instances. This is done by implementing special methods, also known as magic methods, in the class.

Each operator has a corresponding magic method, which is automatically invoked when the operator is used. For example, `+` invokes the `__add__()` method, `-` invokes `__sub__()`, and so on. This feature allows custom objects to behave like built-in types in arithmetic, comparison, and other operations.

<h2>Unary Operators</h2>

Unary operators that operate on a single operand. In Python, unary operators include `-` (unary minus), `+` (unary plus) and `~` (bitwise NOT). 

`-` implemented with `__neg__`. Unary arithmetic minus. If `x` is `-2`, then `-x == 2`

In [1]:
class Number:
    def __init__(self, value):
        self.value = value

    def __neg__(self):
        return Number(-self.value)


x = Number(-2)
(-x).value

2

`+` implemeted with `__pos__`. Unary arithmetic plus. Normally `x == +x`, but there are a few special cases where this is incorrect.

In [2]:
class Number:
    def __init__(self, value):
        self.value = value

    def __pos__(self):
        return Number(+self.value)


x = Number(2)
(+x).value

2

`~` implemetned with `__invert__`. Bitwise inversion of an integer, defined as `~x == -(x + 1)`. If `x` is `2`, then `~x == -3`.

In [3]:
class Number:
    def __init__(self, value):
        self.value = value

    def __invert__(self):
        return Number(~self.value)


x = Number(2)
(~x).value

-3

<h3>When `x` is not equal to `+x`</h3>

Changing precision in an arithmetic context can cause `x` to be different from `+x`

In [4]:
import decimal

# Get a reference to the current global arithmetic context
ctx = decimal.getcontext()
# Set the precision of the arithmetic context to 40
ctx.prec = 40
# Calculate 1/3 with the current precision
one_third = decimal.Decimal('1') / decimal.Decimal('3')
# Print the result - 40 digits after the dot
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [5]:
one_third == +one_third

True

In [6]:
# Decrease precision to 28 - the default value for the Decimal class
ctx.prec = 28
one_third == +one_third

False

In [7]:
+one_third

Decimal('0.3333333333333333333333333333')

Unary `+` generates a new `Counter` object, which does not include elements with zero and negative counters

In [8]:
from collections import Counter

ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [9]:
ct['r'] = -3
ct['d'] = 0
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})

In [10]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

<h2>Overloading of the Vector Addition Operator <code>+</code></h2>

This `Vector` class represents a multidimensional vector with support for several commomn operations.

In [11]:
import functools
import itertools
import math
import operator
import reprlib
from array import array
from collections import abc


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except TypeError:
            return NotImplemented
        return Vector(n * factor for n in self)

    def __rmul__(self, scalar):
        return self * scalar

    def __matmul__(self, other):
        if isinstance(other, abc.Sized) and isinstance(other, abc.Iterable):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other

The addition of two vectors gives a new vector whose components are sums of the corresponding components of the summands

In [12]:
v1 = Vector([1, 2, 3])
v2 = Vector([4, 5, 6])

v1 + v2

Vector([5.0, 7.0, 9.0])

In [13]:
v1 + v2 == Vector([1 + 4, 2 + 5, 3 + 6])

True

Add two `Vector` instances of different lengths

In [14]:
v1 = Vector([1, 2, 3, 4])
v2 = Vector([5, 6])

v1 + v2

Vector([6.0, 8.0, 3.0, 4.0])

Addition with objects other than `Vector`

In [15]:
v1 = Vector([1, 2, 3])
v1 + (10, 20, 30)

Vector([11.0, 22.0, 33.0])

The `+` operator's `__add__` and `__radd__` methods are safely overloaded. The `__radd__` method simply has its work delegated to the `__add__` method. If the operator's special method cannot return the correct result due to type incompatibility, `NotImplemented` should be returned instead of raising a `TypeError` exception.

In [16]:
def __add__(self, other):
    try:
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs)
    except TypeError:
        return NotImplemented


def __radd__(self, other):
    return self + other

<h2>Multiplication Operator Overload on a Scalar <code>*</code></h2>

In [17]:
v1 = Vector([1, 2, 3])

v1 * 10

Vector([10.0, 20.0, 30.0])

In [18]:
11 * v1

Vector([11.0, 22.0, 33.0])

Multiplication operations work if the operand types are compatible. The `scalar` argument must be a number that when multiplied by `float` gives `float` (because the internal represantation of the `Vector` class uses an array of numbers of type `float`) 

In [19]:
def __mul__(self, scalar):
    try:
        factor = float(scalar)
    except TypeError:
        return NotImplemented
    return Vector(n * factor for n in self)


def __rmul__(self, scalar):
    return self * scalar

If the `scalar` cannot be converted to a `float`, it is not known how to process it, so `NotImplemented` is returned to allow Python to try the `scalar` operand's `__rmul__` operator. The `__rmul__` method simply has the product of `self * scalar` computed, with all the work delegated to the `__mul__` method.

Multiplication of vectors by `scalar` values of common and not-so-common numeric types

In [20]:
v1 = Vector([1.0, 2.0, 3.0])

14 * v1

Vector([14.0, 28.0, 42.0])

In [21]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [22]:
from fractions import Fraction

v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])